In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

**Initializing the Weights and Bias configuration.**

In [2]:
sweep_config = {

    'method' : 'grid',
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'
                },
    'parameters' : {
        'batch_size': {'values': [8,16]},
        'learning_rate': {'values': [0.001,0.0001]},
        'hidden_nodes': {'values': [128,64]},
        'img_size': {'values': [16, 224]},
        'epochs': {'values': [5,10]}
    }
}


sweep_id = wandb.sweep(sweep_config, project="5-flowers-classification")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: tard0exk
Sweep URL: https://wandb.ai/ayoubaamir0201-ensam-casablanca/5-flowers-classification/sweeps/tard0exk


In [3]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(config.hidden_nodes, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5)]
      )

In [4]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: tmnn7l31 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: ayoubaamir0201 (ayoubaamir0201-ensam-casablanca) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    413/Unknown 145s 348ms/step - accuracy: 0.3253 - loss: 1.4895

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

batch/accuracy,▁▂▃▄▄▆▅▆▆▆▆▁▁▆▅▅▄▆▇▅▅▅▅▅█▄▄▃▇▇▅▅▄▅▃▄▇▆▅▄
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▄█▁▅▅▆▄▅▃▃
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁
epoch/val_accuracy,█▁█▆▇▅▅▆▅▅
epoch/val_loss,█▂▁▁▁▁▁▁▁▁
batch/accuracy,0.27063


wandb: Agent Starting Run: di01nbdb with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.0001


Epoch 1/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 141s 678ms/step - accuracy: 0.2973 - loss: 19.5005 - val_accuracy: 0.2865 - val_loss: 4.7346
Epoch 2/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 137s 662ms/step - accuracy: 0.3711 - loss: 4.4876 - val_accuracy: 0.3432 - val_loss: 6.0303
Epoch 3/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 138s 665ms/step - accuracy: 0.4012 - loss: 3.2210 - val_accuracy: 0.2568 - val_loss: 1.6763
Epoch 4/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 141s 682ms/step - accuracy: 0.2451 - loss: 1.6061 - val_accuracy: 0.3054 - val_loss: 1.6908
Epoch 5/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 140s 675ms/step - accuracy: 0.3199 - loss: 1.5765 - val_accuracy: 0.2351 - val_loss: 1.7167
Epoch 6/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 144s 695ms/step - accuracy: 0.2392 - loss: 1.5924 - val_accuracy: 0.2351 - val_loss: 1.6616
Epoch 7/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 137s 662ms/step - accuracy: 0.2493 - loss: 1.5745 - val_accuracy: 0.2676 - val_loss: 1.6577
Epoch 8/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 137s 661ms/step - accuracy: 0.2887 

batch/accuracy,▂▃▄▄▄▆▆▆██▆▂▂▂▂▄▃▃▁▁▁▂▂▂▁▂▂▂▂▃▃▃▂▂▂▂▂▂▂▂
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▃▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▅█▇▂▃▁▂▃▁▂
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▁▁▁▁▁
epoch/val_accuracy,▅█▄▆▃▃▅▃▁▂
epoch/val_loss,▆█▁▁▁▁▁▁▁▁
batch/accuracy,0.2588


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


**The 5-Flowers-Classification experimental report Link :**
https://api.wandb.ai/links/ayoubaamir0201-ensam-casablanca/tee8topc